In [ ]:
!nvidia-smi

In [ ]:
!pip install keras_self_attention

In [ ]:
import pickle
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras_self_attention import SeqSelfAttention
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras import backend as K
import pandas as pd

In [ ]:
features = 1280
# 2048 for ResNet
# 1024 for DenseNet
# 1280 for MobileNet
output_file = '/content/drive/MyDrive/MobileNetV2_Results_.p'


with open ('/content/drive/MyDrive//saveX_MobileNet.p', 'rb') as fp:
  X = pickle.load(fp)
    
with open ('/content/drive/MyDrive/saveY_MobileNet.p', 'rb') as fp:
  Y = pickle.load(fp)

In [ ]:
def pad_into_array(list, total_length):
  T = []
    
  for i in range(len(list)):
    pad = total_length - X[i].shape[0]
    if pad <= 0:
      T.append(list[i][:total_length, :])
    else:
      T.append(np.pad(list[i], [(0, pad), (0, 0)]))
  
  return np.array(T)

In [ ]:
def print_plots(history, neurons, depth, model_type):

  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'valid'], loc='upper left')
  plt.savefig('/content/drive/MyDrive/Type{}_Depth{}_Neurons{}_Acc.png'.format(model_type, depth, neurons))
  plt.clf()

  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'valid'], loc='upper left')
  plt.savefig('/content/drive/MyDrive/Type{}_Depth{}_Neurons{}_Loss.png'.format(model_type, depth, neurons))
  plt.clf()

In [ ]:
max_length = 30

X = pad_into_array(X, max_length)

print(type(X))
print(X.shape)
print(type(Y))
print(Y.shape)

<class 'numpy.ndarray'>
(1600, 30, 1280)
<class 'numpy.ndarray'>
(1600, 11)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=22)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1280, 30, 1280)
(1280, 11)
(320, 30, 1280)
(320, 11)


In [ ]:
total_neurons = [32, 64, 128, 256]
output_classes = 11

results = pd.DataFrame(columns = ['Type', 'Depth', 'Neurons', 'Accuracy', 'Loss'])
print(results)

Empty DataFrame
Columns: [Type, Depth, Neurons, Accuracy, Loss]
Index: []


In [ ]:
for neurons in total_neurons:
    
  print('Doing Type 1, Depth 1, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.LSTM(neurons))
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '1(LSTM)', 'Depth' : 1, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 1, 1)
    
  K.clear_session()
  
  print('Doing Type 1, Depth 2, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.LSTM(neurons, return_sequences=True))
  model.add(layers.LSTM(neurons))
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '1(LSTM)', 'Depth' : 2, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 2, 1)
    
  K.clear_session()
  
  print('Doing Type 1, Depth 3, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.LSTM(neurons, return_sequences=True))
  model.add(layers.LSTM(neurons, return_sequences=True))
  model.add(layers.LSTM(neurons))
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '1(LSTM)', 'Depth' : 3, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 3, 1)
    
  K.clear_session()

Doing Type 1, Depth 1, Neuron 32
Doing Type 1, Depth 2, Neuron 32
Doing Type 1, Depth 3, Neuron 32
Doing Type 1, Depth 1, Neuron 64
Doing Type 1, Depth 2, Neuron 64
Doing Type 1, Depth 3, Neuron 64
Doing Type 1, Depth 1, Neuron 128
Doing Type 1, Depth 2, Neuron 128
Doing Type 1, Depth 3, Neuron 128
Doing Type 1, Depth 1, Neuron 256
Doing Type 1, Depth 2, Neuron 256
Doing Type 1, Depth 3, Neuron 256


<Figure size 432x288 with 0 Axes>

In [ ]:
for neurons in total_neurons:
    
  print('Doing Type 2, Depth 1, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.Bidirectional(layers.LSTM(neurons)))
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '2(BiLSTM)', 'Depth' : 1, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 1, 2)
    
  K.clear_session()
    
  print('Doing Type 2, Depth 2, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(neurons)))
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '2(BiLSTM)', 'Depth' : 2, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 2, 2)
    
  K.clear_session()
  
  print('Doing Type 2, Depth 3, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(neurons)))
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '2(BiLSTM)', 'Depth' : 3, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 3, 2)
    
  K.clear_session()

Doing Type 2, Depth 1, Neuron 32
Doing Type 2, Depth 2, Neuron 32
Doing Type 2, Depth 3, Neuron 32
Doing Type 2, Depth 1, Neuron 64
Doing Type 2, Depth 2, Neuron 64
Doing Type 2, Depth 3, Neuron 64
Doing Type 2, Depth 1, Neuron 128
Doing Type 2, Depth 2, Neuron 128
Doing Type 2, Depth 3, Neuron 128
Doing Type 2, Depth 1, Neuron 256
Doing Type 2, Depth 2, Neuron 256
Doing Type 2, Depth 3, Neuron 256


<Figure size 432x288 with 0 Axes>

In [ ]:
for neurons in total_neurons:
    
  print('Doing Type 3, Depth 1, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(SeqSelfAttention(attention_activation='sigmoid'))
  model.add(layers.Flatten())
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '3(AttBiLSTM)', 'Depth' : 1, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 1, 3)
    
  K.clear_session()
    
  print('Doing Type 3, Depth 2, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(SeqSelfAttention(attention_activation='sigmoid'))
  model.add(layers.Flatten())
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '3(AttBiLSTM)', 'Depth' : 2, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 2, 3)
    
  K.clear_session()
  
  print('Doing Type 3, Depth 3, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(SeqSelfAttention(attention_activation='sigmoid'))
  model.add(layers.Flatten())
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '3(AttBiLSTM)', 'Depth' : 3, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 3, 3)
    
  K.clear_session()

Doing Type 3, Depth 1, Neuron 32
Doing Type 3, Depth 2, Neuron 32
Doing Type 3, Depth 3, Neuron 32
Doing Type 3, Depth 1, Neuron 64
Doing Type 3, Depth 2, Neuron 64
Doing Type 3, Depth 3, Neuron 64
Doing Type 3, Depth 1, Neuron 128
Doing Type 3, Depth 2, Neuron 128
Doing Type 3, Depth 3, Neuron 128
Doing Type 3, Depth 1, Neuron 256
Doing Type 3, Depth 2, Neuron 256
Doing Type 3, Depth 3, Neuron 256


<Figure size 432x288 with 0 Axes>

In [ ]:
with open(output_file, 'wb') as fp:
    pickle.dump(results, fp)